# GameBus Health Behavior Mining - Data Extraction

This notebook demonstrates how to extract data from the GameBus platform using the framework.

## Setup

First, let's set up our environment and import the necessary modules.

In [14]:
import sys
import os
import pandas as pd
import json
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Add the project root directory to the Python path
sys.path.append('..')

# Import project modules
from config.credentials import AUTHCODE
from config.paths import USERS_FILE_PATH, RAW_DATA_DIR
from src.extraction.gamebus_client import GameBusClient
from src.extraction.data_collectors import (
    LocationDataCollector, 
    MoodDataCollector,
    ActivityTypeDataCollector,
    HeartRateDataCollector,
    AccelerometerDataCollector,
    NotificationDataCollector
)
from src.utils.logging import setup_logging
from src.utils.file_handlers import load_json, save_json, load_csv, save_csv

## Initialize Logging

Set up logging for the notebook.

In [15]:
logger = setup_logging(log_level="INFO")
logger.info("Notebook initialized")

2025-05-06 15:56:11,182 - gamebus_health_mining - INFO - Notebook initialized
2025-05-06 15:56:11,182 - gamebus_health_mining - INFO - Notebook initialized


## Load GameBus Users

Load the GameBus users from the CSV file. If the file doesn't exist yet, we'll create it from the example file.

In [16]:
# Load the users
try:
    users_df = pd.read_csv(USERS_FILE_PATH, delimiter=';')
    display(users_df.head())
    print(f"Loaded {len(users_df)} users")
except Exception as e:
    # If users file doesn't exist yet, we need to create it
    print(f"Failed to load users file: {e}")
    print("Loading example users from the examples directory...")
    example_users_path = os.path.join('..', 'examples', 'GB-users.csv')
    users_df = pd.read_csv(example_users_path, delimiter=';')
    # Save to configured path
    os.makedirs(os.path.dirname(USERS_FILE_PATH), exist_ok=True)
    users_df.to_csv(USERS_FILE_PATH, sep=';', index=False)
    display(users_df.head())
    print(f"Loaded {len(users_df)} example users and saved to {USERS_FILE_PATH}")

,Username,Password
0,info+bm@gamebus.nl,password1234
1,m.i.da.graca.jorge.da.silva.ribeiro+test@tue.nl,password1234


Loaded 2 users


## Initialize GameBus Client

Create a GameBus client for API interactions.

In [17]:
# Initialize the GameBus client
client = GameBusClient(AUTHCODE)
print("GameBus client initialized")

GameBus client initialized


## Select User

Select a user from the list to extract data for.

In [18]:
# Select a user
user_index = 0  # Change this to select a different user
selected_user = users_df.iloc[user_index]
username = selected_user['Username']
password = selected_user['Password']

print(f"Selected user: {username}")

Selected user: info+bm@gamebus.nl


## Authenticate User

Authenticate with the GameBus API and get the player token and ID.

In [19]:
# Get player token and ID
token = client.get_player_token(username, password)
player_id = client.get_player_id(token)

print(f"Player token: {token[:10]}..." if token else "Failed to get token")
print(f"Player ID: {player_id}" if player_id else "Failed to get player ID")

Player token: eyJhbGciOi...
Player ID: 107631


## Extract GPS Location Data

Extract GPS location data from GameBus. This includes latitude, longitude, altitude, speed, error, timestamp, and arm.

In [20]:
# Create a location data collector
location_collector = LocationDataCollector(client, token, player_id)

# Collect location data
location_data, location_file = location_collector.collect()

print(f"Collected {len(location_data)} location data points")
print(f"Data saved to {location_file}")

# Display a sample of the data
location_df = pd.DataFrame(location_data)
if len(location_df) > 0:
    display(location_df.head())

Collected 9853 location data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_location.json


,LATITUDE,LONGITUDE,ALTIDUDE,SPEED,ERROR,TIMESTAMP,ARM,activity_id,date,gameDescriptor
0,52.09715270996094,5.109695911407471,32.20000076293945,3.852000188827515,29,1745804145,Arm 2,3312956,1745804205000,GEOFENCE
1,52.09716796875,5.109516143798828,33.70000076293945,0,21,1745804085,Arm 2,3312955,1745804145000,GEOFENCE
2,52.09715270996094,5.109503746032715,32.09999847412109,2.016000008583069,19,1745804025,Arm 2,3312953,1745804085000,GEOFENCE
3,52.09713745117188,5.109537601470947,28.10000038146973,0,17,1745803965,Arm 2,3312952,1745804025000,GEOFENCE
4,52.09712219238281,5.109561443328857,18.89999961853027,0,19,1745803905,Arm 2,3312951,1745803965000,GEOFENCE


## Extract Mood Data

Extract mood logging data from GameBus. This includes valence_state_value, arousal_state_value, stress_state_value, and event_timestamp.

In [21]:
# Create a mood data collector
mood_collector = MoodDataCollector(client, token, player_id)

# Collect mood data
mood_data, mood_file = mood_collector.collect()

print(f"Collected {len(mood_data)} mood data points")
print(f"Data saved to {mood_file}")

# Display a sample of the data
mood_df = pd.DataFrame(mood_data)
if len(mood_df) > 0:
    display(mood_df.head())

Collected 23 mood data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_mood.json


,VALENCE_STATE_VALUE,AROUSAL_STATE_VALUE,STRESS_STATE_VALUE,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,5,5,5,1745773701931,3308348,1746041731000,LOG_MOOD
1,5,5,5,1745773655824,3308316,1746040763000,LOG_MOOD
2,6,7,8,1744385153147,3281465,1745862466000,LOG_MOOD
3,6,6,6,1744374335208,3279901,1745853529000,LOG_MOOD
4,5,6,8,1744360675271,3279521,1745809376000,LOG_MOOD


## Extract Activity Type Data

Extract activity type data from GameBus. This includes src, ts (timestamp), type, speed, steps, walks, runs, freq, distance, and cals.

In [22]:
# Create an activity type data collector
activity_collector = ActivityTypeDataCollector(client, token, player_id)

# Collect activity type data
activity_data, activity_file = activity_collector.collect()

print(f"Collected {len(activity_data)} activity data points")
print(f"Data saved to {activity_file}")

# Display a sample of the data
activity_df = pd.DataFrame(activity_data)
if len(activity_df) > 0:
    display(activity_df.head())

Collected 970 activity data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_activity_type.json


,src,ts,type,speed,steps,walks,runs,freq,distance,cals,activity_id,activity_date
0,p,1745804099854,NOT_MOVING,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3312954,1745804100000
1,p,1745802899849,NOT_MOVING,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3311118,1745802900000
2,p,1745801699815,NOT_MOVING,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3311065,1745801700000
3,p,1745800499819,NOT_MOVING,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3310975,1745800500000
4,p,1745799299823,NOT_MOVING,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3310884,1745799300000


## Extract Heart Rate Data

Extract heart rate monitoring data from GameBus. This includes ts (timestamp), hr (heartrate), and pp.

In [23]:
# Create a heart rate data collector
heartrate_collector = HeartRateDataCollector(client, token, player_id)

# Collect heart rate data
heartrate_data, heartrate_file = heartrate_collector.collect()

print(f"Collected {len(heartrate_data)} heart rate data points")
print(f"Data saved to {heartrate_file}")

# Display a sample of the data
heartrate_df = pd.DataFrame(heartrate_data)
if len(heartrate_df) > 0:
    display(heartrate_df.head())

Collected 499047 heart rate data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_heartrate.json


,ts,hr,pp,activity_id,activity_date
0,1745804267223,-3,0,3312958,1745804326000
1,1745804267303,-3,0,3312958,1745804326000
2,1745804267382,-3,0,3312958,1745804326000
3,1745804267463,-3,0,3312958,1745804326000
4,1745804267543,-3,0,3312958,1745804326000


## Extract Accelerometer Data

Extract accelerometer data from GameBus. This includes ts (timestamp), x axis, y axis, and z axis.

In [24]:
# Create an accelerometer data collector
accelerometer_collector = AccelerometerDataCollector(client, token, player_id)

# Collect accelerometer data
accelerometer_data, accelerometer_file = accelerometer_collector.collect()

print(f"Collected {len(accelerometer_data)} accelerometer data points")
print(f"Data saved to {accelerometer_file}")

# Display a sample of the data
accelerometer_df = pd.DataFrame(accelerometer_data)
if len(accelerometer_df) > 0:
    display(accelerometer_df.head())

Collected 1232770 accelerometer data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_accelerometer.json


,ts,x,y,z,activity_id,activity_date
0,1745804266692,-0.181855,-0.244068,9.659825,3312957,1745804326000
1,1745804266704,-0.191426,-0.279960,9.731609,3312957,1745804326000
2,1745804266706,-0.172283,-0.244068,9.667004,3312957,1745804326000
3,1745804266707,-0.165105,-0.277567,9.707682,3312957,1745804326000
4,1745804266707,-0.244068,-0.248854,9.714860,3312957,1745804326000


## Extract Notification Data

Extract notification data from GameBus. This includes action (e.g., received, read) and event_timestamp.

In [25]:
# Create a notification data collector
notification_collector = NotificationDataCollector(client, token, player_id)

# Collect notification data
notification_data, notification_file = notification_collector.collect()

print(f"Collected {len(notification_data)} notification data points")
print(f"Data saved to {notification_file}")

# Display a sample of the data
notification_df = pd.DataFrame(notification_data)
if len(notification_df) > 0:
    display(notification_df.head())

Collected 64 notification data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_notifications.json


,ACTION,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,RECEIVED,1745776500048,3308408,1745776500000,NOTIFICATION(DETAIL)
1,READ,1745773698947,3308346,1745773699000,NOTIFICATION(DETAIL)
2,READ,1745773650939,3308314,1745773651000,NOTIFICATION(DETAIL)
3,RECEIVED,1744482991379,3308255,1744482991000,NOTIFICATION(DETAIL)
4,RECEIVED,1744472101194,3307574,1744472101000,NOTIFICATION(DETAIL)


## Summary

Let's summarize all the data we've collected.

In [26]:
# Create a summary of collected data
summary = {
    "Location": len(location_data),
    "Mood": len(mood_data),
    "Activity": len(activity_data),
    "Heart Rate": len(heartrate_data),
    "Accelerometer": len(accelerometer_data),
    "Notification": len(notification_data)
}

summary_df = pd.DataFrame({
    "Data Type": list(summary.keys()),
    "Count": list(summary.values())
})

# Display summary
display(summary_df)

print(f"\nTotal data points collected: {sum(summary.values())}")

,Data Type,Count
0,Location,9853
1,Mood,23
2,Activity,970
3,Heart Rate,499047
4,Accelerometer,1232770
5,Notification,64



Total data points collected: 1742727


## Next Steps

Now that we have extracted the raw data from GameBus, the next steps would be:

1. **Preprocessing**: Clean and normalize the data, handle missing values, synchronize timestamps, etc.
2. **Activity Recognition**: Recognize human activities from the sensor data using machine learning techniques.
3. **OCEL Generation**: Transform the preprocessed data and recognized activities into the Object-Centric Event Log (OCEL) format for process mining.

These steps will be covered in the subsequent notebooks.